In [1]:
import pandas as pd
import numpy as np
df1=pd.read_csv('F:/0Sem 7/ML Lab/amazon food review dataset/Reviews.csv')
df1.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [2]:
score=df1.values[:,6]
text=df1.values[:,9]
reviews=np.vstack((score,text)).T
print(score.shape, text.shape, reviews.shape)

(568454,) (568454,) (568454, 2)


In [3]:
p=0
n=0
for i in range(reviews.shape[0]):
    if reviews[i,0] > 3:
        reviews[i,0]=0 #positive review
        p=p+1
    else:
        reviews[i,0]=1 #negative review
        n=n+1

In [4]:
reviews = reviews[reviews[:,0].argsort()]   #sort by 1st column

In [5]:
train=[]
for i in range(5000):
    train.append(reviews[i])
    
for i in range(443777,443777+5000):
    train.append(reviews[i])
    
train=np.asarray(train)
train.shape

(10000, 2)

In [6]:
my_reviews1=np.array([0,'This is a very good product. I am very happy with this item.'])
my_reviews2=np.array([1,'The product is very bad. I am very unsatisfied with the appearance.'])
my_reviews3=np.array([0,'It was one if the best items i have purchased. Very good.'])
my_reviews4=np.array([0,'All members of my family enjoyed the item. It is well thought.'])
my_reviews5=np.array([1,'Extremely poor quality. I hated the item and so did my brothers.'])

#train=np.vstack((train,my_reviews1))
#train=np.vstack((train,my_reviews2))
#train=np.vstack((train,my_reviews3))
#train=np.vstack((train,my_reviews4))
#train=np.vstack((train,my_reviews5))

In [7]:
import random as r
test=[]
for i in range(2000):
    index=r.randint(0,reviews.shape[0])
    test.append(reviews[index])
    
test=np.asarray(test)
test.shape

(2000, 2)

In [8]:
#test=np.vstack((test,my_reviews1))
#test=np.vstack((test,my_reviews2))
#test=np.vstack((test,my_reviews3))
#test=np.vstack((test,my_reviews4))
#test=np.vstack((test,my_reviews5))

In [9]:
train_all_words=[]
for i in range(train.shape[0]):
    train_all_words.append(train[i,1].split())
    
train_all_words = [item for sublist in train_all_words for item in sublist]

test_all_words=[]
for i in range(test.shape[0]):
    test_all_words.append(test[i,1].split())
    
test_all_words = [item for sublist in test_all_words for item in sublist]

from collections import Counter
def common_words(words, number_of_words, reverse=False):
    counter = Counter(words)
    return sorted(counter, key = counter.get, reverse=reverse)[:number_of_words]

train_least_common=common_words(train_all_words,200)
train_most_common=common_words(train_all_words,200,reverse=True)

test_least_common=common_words(test_all_words,200)
test_most_common=common_words(test_all_words,200,reverse=True)

In [10]:
for i in range(train.shape[0]):
    train[i,1]=train[i,1].split()
    
for i in range(test.shape[0]):
    test[i,1]=test[i,1].split()

In [11]:
for i in range(train.shape[0]):
    for item in train_most_common:
        if item in train[i,1]:
            train[i,1].remove(item)
    for item in train_least_common:
        if item in train[i,1]:
            train[i,1].remove(item)
            
for i in range(test.shape[0]):
    for item in test_most_common:
        if item in test[i,1]:
            test[i,1].remove(item)
    for item in test_least_common:
        if item in test[i,1]:
            test[i,1].remove(item)

In [12]:
for i in range(train.shape[0]):
    train[i,1]=" ".join(train[i,1])
    
for i in range(test.shape[0]):
    test[i,1]=" ".join(test[i,1])

In [13]:
train_docs=[]
test_docs=[]
train_label=[]
test_label=[]
for i in range(train.shape[0]):
    train_docs.append(train[i,1])
    train_label.append(train[i,0])
    
for i in range(test.shape[0]):
    test_docs.append(test[i,1])
    test_label.append(test[i,0])

In [14]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(train_docs)
vocab_size = len(t.word_index) + 1
#integer encode the documents
encoded_docs = t.texts_to_sequences(train_docs)

#print(encoded_docs)
#pad documents to a max length of 4 words

max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

#print(padded_docs)
#load the whole embedding into memory

embeddings_index = dict()
f = open('F:/0Sem 7/ML Lab/glove/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

#print('Loaded %s word vectors.' % len(embeddings_index))

#create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Using TensorFlow backend.


In [15]:
embedding_matrix.shape

(19814, 100)

In [16]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#define model
model = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model.add(e)
model.add(layers.LSTM(64, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

In [17]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1981400   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,023,705
Trainable params: 42,305
Non-trainable params: 1,981,400
_________________________________________________________________
None


In [18]:
for layer in model.layers:
    print(layer.output_shape)

(None, 100, 100)
(None, 64)
(None, 64)
(None, 1)


In [19]:
train_label=np.asarray(train_label)
train_label.shape

(10000,)

In [20]:
padded_docs.shape

(10000, 100)

In [21]:
# prepare tokenizer
t1 = Tokenizer()
t1.fit_on_texts(test_docs)
vocab_size1 = len(t1.word_index) + 1
#integer encode the documents
encoded_docs1 = t1.texts_to_sequences(test_docs)

#print(encoded_docs)
#pad documents to a max length of 4 words

max_length1 = 100
padded_docs1 = pad_sequences(encoded_docs1, maxlen=max_length1, padding='post')
test_label=np.asarray(test_label)
#print(padded_docs)

In [22]:
# fit the model
model.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=50, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/50
10000/10000 [==============================] - 44s 4ms/sample - loss: 0.7321 - accuracy: 0.5237 - val_loss: 0.6946 - val_accuracy: 0.7195
Epoch 2/50
10000/10000 [==============================] - 36s 4ms/sample - loss: 0.8181 - accuracy: 0.5321 - val_loss: 0.6902 - val_accuracy: 0.7300
Epoch 3/50
10000/10000 [==============================] - 36s 4ms/sample - loss: 0.6892 - accuracy: 0.5289 - val_loss: 0.6926 - val_accuracy: 0.7140
Epoch 4/50
10000/10000 [==============================] - 36s 4ms/sample - loss: 0.6883 - accuracy: 0.5290 - val_loss: 0.6959 - val_accuracy: 0.7105
Epoch 5/50
10000/10000 [==============================] - 36s 4ms/sample - loss: 1182558.7550 - accuracy: 0.5261 - val_loss: 0.6851 - val_accuracy: 0.7265
Epoch 6/50
10000/10000 [==============================] - 36s 4ms/sample - loss: 0.6897 - accuracy: 0.5301 - val_loss: 0.6931 - val_accuracy: 0.7120
Epoch 7/50
10000/10000 [===========================

In [25]:
"2nd LSTM Model"

'2nd LSTM Model'

In [27]:
#define model
model1 = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model1.add(e)
model1.add(layers.LSTM(64, activation='tanh', return_sequences=True))
model1.add(layers.LSTM(128, activation='tanh', return_sequences=False))
model1.add(layers.Flatten())
model1.add(layers.Dense(1, activation='sigmoid'))

# compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          1981400   
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 64)           42240     
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               98816     
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,122,585
Trainable params: 141,185
Non-trainable params: 1,981,400
_________________________________________________________________
None


In [31]:
# fit the model
history1=model1.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=50, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/50
10000/10000 [==============================] - 22s 2ms/sample - loss: 0.6915 - accuracy: 0.5237 - val_loss: 0.7199 - val_accuracy: 0.2175
Epoch 2/50
10000/10000 [==============================] - 8s 772us/sample - loss: 0.6878 - accuracy: 0.5327 - val_loss: 0.7133 - val_accuracy: 0.2170
Epoch 3/50
10000/10000 [==============================] - 8s 763us/sample - loss: 0.6899 - accuracy: 0.5167 - val_loss: 0.6976 - val_accuracy: 0.7000
Epoch 4/50
10000/10000 [==============================] - 8s 771us/sample - loss: 0.6875 - accuracy: 0.5399 - val_loss: 0.6200 - val_accuracy: 0.6810
Epoch 5/50
10000/10000 [==============================] - 8s 782us/sample - loss: 0.6728 - accuracy: 0.5756 - val_loss: 0.5747 - val_accuracy: 0.7820
Epoch 6/50
10000/10000 [==============================] - 8s 781us/sample - loss: 0.6917 - accuracy: 0.5137 - val_loss: 0.6782 - val_accuracy: 0.7025
Epoch 7/50
10000/10000 [============================

In [32]:
"3rd LSTM Model"

'3rd LSTM Model'

In [33]:
#define model
model2 = Sequential()
e = tf.keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model2.add(e)
model2.add(layers.LSTM(64, activation='relu', return_sequences=True))
model2.add(layers.LSTM(128, activation='relu', return_sequences=False))
model2.add(layers.Flatten())
model2.add(layers.Dense(1, activation='sigmoid'))

# compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model2.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          1981400   
_________________________________________________________________
lstm_7 (LSTM)                (None, 100, 64)           42240     
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               98816     
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,122,585
Trainable params: 141,185
Non-trainable params: 1,981,400
_________________________________________________________________
None


In [34]:
# fit the model
history2=model2.fit(padded_docs, train_label, validation_data=(padded_docs1,test_label), epochs=50, verbose=1)

Train on 10000 samples, validate on 2000 samples
Epoch 1/50
10000/10000 [==============================] - 84s 8ms/sample - loss: 743415.0260 - accuracy: 0.5231 - val_loss: 0.6745 - val_accuracy: 0.7830
Epoch 2/50
10000/10000 [==============================] - 83s 8ms/sample - loss: 0.6986 - accuracy: 0.5113 - val_loss: 0.6883 - val_accuracy: 0.7220
Epoch 3/50
10000/10000 [==============================] - 83s 8ms/sample - loss: 0.7044 - accuracy: 0.5229 - val_loss: 0.6817 - val_accuracy: 0.7145
Epoch 4/50
10000/10000 [==============================] - 83s 8ms/sample - loss: 0.6919 - accuracy: 0.5257 - val_loss: 0.6896 - val_accuracy: 0.7000
Epoch 5/50
10000/10000 [==============================] - 82s 8ms/sample - loss: 1.1787 - accuracy: 0.5216 - val_loss: 0.7101 - val_accuracy: 0.2170
Epoch 6/50
10000/10000 [==============================] - 82s 8ms/sample - loss: 0.6919 - accuracy: 0.5039 - val_loss: 0.6997 - val_accuracy: 0.2195
Epoch 7/50
10000/10000 [============================